# Introduction

In urban safety planning in Japan, it's crucial to analyze which areas of Japan need extra earthquake reinforcement, which areas are both high in population density and prone to earthquakes, etc

<center>
<img src="https://i.imgur.com/Kuh9gPj.png" width="450"><br/>
</center>

Before we get started, run the code cell below to set everything up.

In [ ]:
import pandas as pd
import geopandas as gpd

import folium
from folium import Choropleth
from folium.plugins import HeatMap

from learntools.core import binder
binder.bind(globals())
from learntools.geospatial.ex3 import *

We can use the `embed_map()` function to visualize my maps.

In [ ]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

### The first map we want to visualize is how does earthquakes conincide with plate boundaries. 

We get the shp files of plate boundaries of Japan, and create a DataFrame `plate_boundaries` that shows global plate boundaries. The "coordinates" column is a list of (latitude, longitude) locations along the boundaries.

In [ ]:
plate_boundaries = gpd.read_file("../input/geospatial-learn-course-data/Plate_Boundaries/Plate_Boundaries/Plate_Boundaries.shp")
plate_boundaries['coordinates'] = plate_boundaries.apply(lambda x: [(b,a) for (a,b) in list(x.geometry.coords)], axis='columns')
plate_boundaries.drop('geometry', axis=1, inplace=True)

plate_boundaries.head()

In [ ]:
plate_boundaries['coordinates'][0]

Next, we load the historical earthquake data into a DataFrame `earthquakes`.

In [ ]:
# Load the data and print the first 5 rows
earthquakes = pd.read_csv("../input/geospatial-learn-course-data/earthquakes1970-2014.csv", parse_dates=["DateTime"])
earthquakes.head()

In [ ]:
earthquakes_data=earthquakes[["Latitude", "Longitude"]]

In [ ]:
earthquakes_data.head()

We now visualize the plate boundaries on a map.  Use all of the earthquake data to add a heatmap to the same map, to determine whether earthquakes coincide with plate boundaries.  

In [ ]:
# Create a base map with plate boundaries
m_1 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_1)

# Add a heatmap to the map
data=earthquakes_data
m = folium.Map([48, 5], tiles='stamentoner')

HeatMap(data, radius=15).add_to(folium.FeatureGroup(name='Heat Map').add_to(m_1))
folium.LayerControl().add_to(m_1)

# Show the map
embed_map(m_1, 'q_1.html')

From the mao above, we can see that earthquakes coincide with plate boundaries.

### Is there a relationship between earthquake depth and proximity to a plate boundary in Japan?

I recently read that the depth of earthquakes tells us [important information](https://www.usgs.gov/faqs/what-depth-do-earthquakes-occur-what-significance-depth?qt-news_science_products=0#qt-news_science_products) about the structure of the earth.  I'm interested to see if there are any intereresting global patterns, and I'd also like to understand how depth varies in Japan.



In [ ]:
# Create a base map with plate boundaries
m_2 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_2)
    
# visualize earthquake depth
def color_producer(val):
    if val < 50:
        return 'forestgreen'
    elif val < 100:
        return 'darkorange'
    else:
        return 'darkred'

for i in range(0,len(earthquakes)):
    folium.Circle(
        location=[earthquakes.iloc[i]['Latitude'], earthquakes.iloc[i]['Longitude']],
        radius=2000,
        color=color_producer(earthquakes.iloc[i]['Depth'])).add_to(m_2)


# View the map
embed_map(m_2, 'q_2.html')

In the northern half of Japan, it does appear that earthquakes closer to plate boundaries tend to be shallower (and earthquakes farther from plate boundaries are deeper). This pattern is repeated in other locations, such as the western coast of South America. But, it does not hold everywhere (for instance, in China, Mongolia, and Russia).

Next, we want to know which prefectures have high population density.

We can create a GeoDataFrame `prefectures` that contains the geographical boundaries of Japanese prefectures.

In [ ]:
# GeoDataFrame with prefecture boundaries
prefectures = gpd.read_file("../input/geospatial-learn-course-data/japan-prefecture-boundaries/japan-prefecture-boundaries/japan-prefecture-boundaries.shp")
prefectures.set_index('prefecture', inplace=True)
prefectures.head()

In [ ]:
prefectures["geometry"][0]

Note: the polygon is 

In [ ]:
prefectures['geometry'].__geo_interface__

The next code cell creates a DataFrame `stats` containing the population, area (in square kilometers), and population density (per square kilometer) for each Japanese prefecture.  Run the code cell without changes.

In [ ]:
# DataFrame containing population of each prefecture
population = pd.read_csv("../input/geospatial-learn-course-data/japan-prefecture-population.csv")
population.set_index('prefecture', inplace=True)

# Calculate area (in square kilometers) of each prefecture
area_sqkm = pd.Series(prefectures.geometry.to_crs(epsg=32654).area / 10**6, name='area_sqkm')
stats = population.join(area_sqkm)

# Add density (per square kilometer) of each prefecture
stats['density'] = stats["population"] / stats["area_sqkm"]
stats.head()

We can then create a choropleth map to visualize population density.

In [ ]:
# Create a base map
m_3 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

# Create a choropleth map to visualize population density
Choropleth(geo_data=prefectures['geometry'].__geo_interface__,
           data=stats['density'],
           key_on="feature.id",
           fill_color='YlGnBu',
           legend_name='Population density (per square kilometer)'
          ).add_to(m_3)

# View the map
embed_map(m_3, 'q_3.html')

Tokyo, Kanagawa, and Osaka have the highest population density. All of these prefectures are located in central Japan, and Tokyo and Kanagawa are adjacent.

### Which high-density prefecture is prone to high-magnitude earthquakes?
We visualized population density and earthquake magnitude by adding both a choropleth map and a bubble map to the base map.
We can suggest one prefecture that might benefit from earthquake reinforcement. 

In [ ]:
# Create a base map
m_4 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

# Create a choropleth map to visualize population density
Choropleth(geo_data=prefectures['geometry'].__geo_interface__,
           data=stats['density'],
           key_on="feature.id",
           fill_color='BuPu',
           legend_name='Population density (per square kilometer)'
          ).add_to(m_4)

# YAdd a map to visualize earthquake depth
def color_producer(magnitude):
    if magnitude > 6.5 :
        return 'red'
    else:
        return 'green'

# Add a map to visualize earthquake depth
for i in range(0,len(earthquakes)):
    folium.Circle(
        location=[earthquakes.iloc[i]['Latitude'], earthquakes.iloc[i]['Longitude']],
        popup=("{} ({})").format(
            earthquakes.iloc[i]['Magnitude'],
            earthquakes.iloc[i]['DateTime'].year),
        radius=earthquakes.iloc[i]['Magnitude']**5.5,
        color=color_producer(earthquakes.iloc[i]['Magnitude'])).add_to(m_4)


# View the map
embed_map(m_4, 'q_4.html')

While there's no clear, single answer to this question of which prefecture do I recommend for extra earthquake reinforcement, there are a few reasonable options. Tokyo is by far the most densely populated prefecture and has also experienced a number of earthquakes. Osaka is relatively less densely populated, but experienced an earthquake that was relatively stronger than those near Tokyo. And, the long coast of Kanagawa (in addition to its high density and the historical proximity of strong earthquakes) might lead us to worry about the added potential tsunami risk.